In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 10      # number of variables
num_ineq = 10     # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msNonconvex
model = msNonconvex(num_var, num_ineq, timelimit=60)

In [7]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    try:
        xval, objval = model.solve("scip")
        # eval
        params.append(list(b))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model.cal_violation()))
    except:
        params.append(list(b))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/nc_exact_10-10.csv")

  3%|██                                                                              | 26/1000 [01:14<46:09,  2.84s/it]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

  3%|██▍                                                                             | 31/1000 [01:23<35:28,  2.20s/it]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

  7%|█████▌                                                                        | 71/1000 [05:10<1:00:12,  3.89s/it]

...

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

100%|██████████████████████████████████████████████████████████████████████████████▊| 998/1000 [54:53<00:05,  2.79s/it]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [54:58<00:00,  3.30s/it]


          Obj Val  Constraints Viol  Elapsed Time
count  956.000000        956.000000   1000.000000
mean    -0.136008          0.008267      3.297710
std      0.335179          0.157364      4.185454
min     -0.443489          0.000000      0.208956
25%     -0.362479          0.000000      2.102039
50%     -0.261838          0.000000      2.857839
75%     -0.037607          0.000000      3.748860
max      1.706401          4.023182     59.945528
Number of infeasible solution: 3
Number of None values:  44


## Heuristic - Round

In [8]:
from src.heuristic import naive_round

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_heur_rnd_10-10.csv")

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]

ERROR: Unable to clone Pyomo component attribute. Component 'obj' contains an
uncopyable field '_init_sense' (<class
'pyomo.core.base.initializer.ConstantInitializer'>).  Setting field to `None`
on new object


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [18:08<00:00,  1.09s/it]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.182741          0.138803      1.076832
std       0.295827          0.114389      2.666983
min      -0.443489          0.000000      0.577132
25%      -0.372296          0.050033      0.897822
50%      -0.296870          0.118572      0.909813
75%      -0.105505          0.200926      1.017862
max       1.632131          0.701230     60.539031
Number of infeasible solution: 892


## Heuristic - N1

In [10]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    try:
        xval, objval = model_heur.solve("scip")
        # eval
        params.append(list(b))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model_heur.cal_violation()))
    except:
        params.append(list(b))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/nc_heur_n1_10-10.csv")

  3%|██                                                                              | 26/1000 [00:07<04:30,  3.60it/s]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

  3%|██▍                                                                             | 31/1000 [00:09<04:18,  3.75it/s]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

 12%|█████████▌                                                                     | 121/1000 [02:37<03:49,  3.83it/s]

...

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

100%|██████████████████████████████████████████████████████████████████████████████▊| 997/1000 [08:02<00:01,  1.81it/s]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [08:04<00:00,  2.06it/s]


            Obj Val  Constraints Viol  Elapsed Time
count  9.860000e+02        986.000000   1000.000000
mean   8.881740e+03          0.007361      0.484053
std    2.764399e+05          0.135550      3.305921
min   -2.960331e-01          0.000000      0.071335
25%    1.017836e-01          0.000000      0.078972
50%    7.207409e-01          0.000000      0.110050
75%    3.947507e+00          0.000000      0.279603
max    8.680409e+06          2.982027     60.316354
Number of infeasible solution: 3
Number of None values:  14


## Learnable Rounding

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 248.29
Epoch 1, Validation Loss: 83.27
Epoch 2, Validation Loss: 5.60
Epoch 3, Validation Loss: 5.37
Epoch 4, Validation Loss: 3.74
Epoch 5, Validation Loss: 3.47
Epoch 6, Validation Loss: 4.34
Epoch 7, Validation Loss: 3.25
Epoch 8, Validation Loss: 2.87
Epoch 9, Validation Loss: 3.62
Epoch 10, Validation Loss: 2.81
Epoch 11, Validation Loss: 3.34
Epoch 12, Validation Loss: 2.88
Epoch 13, Validation Loss: 2.65
Epoch 14, Validation Loss: 2.55
Epoch 15, Validation Loss: 2.35
Epoch 16, Validation Loss: 2.15
Epoch 17, Validation Loss: 2.13
Epoch 18, Validation Loss: 2.06
Epoch 19, Validation Loss: 1.81
Epoch 20, Validation Loss: 1.74
Epoch 21, Validation Loss: 1.83
Epoch 22, Validation Loss: 1.78
Epoch 23, Validation Loss: 1.77
Epoch 24, Validation Loss: 2.43
Epoch 25, Validation Loss: 1.76
Epoch 26, Validation Loss: 1.63
Epoch 27, Validation Loss: 1.91
Epoch 28, Validation Loss: 1.77
Epoch 29, Validation Loss: 1.51
Epoch 30, Validation Loss: 1.55
Epoch 31, Valid

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lr_10-10.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 221.41it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      0.920736          0.000948      0.002960
std       0.609609          0.008151      0.001443
min       0.147438          0.000000      0.000997
25%       0.461976          0.000000      0.002000
50%       0.729616          0.000000      0.002512
75%       1.151021          0.000000      0.003011
max       3.899173          0.126378      0.012021
Number of infeasible solution: 24


## Learnable Threshold

In [17]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [18]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 32            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [19]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [20]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 253.56
Epoch 1, Validation Loss: 90.71
Epoch 2, Validation Loss: 5.56
Epoch 3, Validation Loss: 4.45
Epoch 4, Validation Loss: 3.14
Epoch 5, Validation Loss: 2.77
Epoch 6, Validation Loss: 2.62
Epoch 7, Validation Loss: 2.46
Epoch 8, Validation Loss: 2.39
Epoch 9, Validation Loss: 2.48
Epoch 10, Validation Loss: 2.64
Epoch 11, Validation Loss: 2.56
Epoch 12, Validation Loss: 2.56
Epoch 13, Validation Loss: 2.31
Epoch 14, Validation Loss: 2.09
Epoch 15, Validation Loss: 2.45
Epoch 16, Validation Loss: 2.18
Epoch 17, Validation Loss: 2.16
Epoch 18, Validation Loss: 2.45
Epoch 19, Validation Loss: 2.32
Epoch 20, Validation Loss: 2.16
Epoch 21, Validation Loss: 2.14
Epoch 22, Validation Loss: 2.01
Epoch 23, Validation Loss: 2.30
Epoch 24, Validation Loss: 1.81
Epoch 25, Validation Loss: 1.80
Epoch 26, Validation Loss: 1.57
Epoch 27, Validation Loss: 1.61
Epoch 28, Validation Loss: 1.61
Epoch 29, Validation Loss: 1.62
Epoch 30, Validation Loss: 1.48
Epoch 31, Valid

In [21]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lt_10-10.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 229.24it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      0.557996          0.000702      0.002831
std       0.567562          0.006450      0.001462
min      -0.123012          0.000000      0.001001
25%       0.156289          0.000000      0.002031
50%       0.399342          0.000000      0.002292
75%       0.744155          0.000000      0.003100
max       3.367784          0.142946      0.013700
Number of infeasible solution: 23
